Instalando pymongo para efetuar a conexão.


In [ ]:
!pip install pymongo[srv,tls]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.1/669.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.3 MB/s eta 0:00:00


Importando as bibliotecas necessárias.

In [ ]:
import pymongo
import json
import pprint
from datetime import datetime

Configurando conexão.

In [ ]:
mongo_user = "yasmincardozo"
mongo_pw = "pucrs2024"
mongo_host = "clusterbd2.osalpzb.mongodb.net"
mongo_app = "ClusterBD2"

In [ ]:
mongo_user = "EduardaPatricio"
mongo_pw = "duds1622"
mongo_host = "clusterbd2.v3hmqat.mongodb.net"
mongo_app = "ClusterBD2"

Formatando a conexão.

YASMIN:

In [ ]:
conn = pymongo.MongoClient("mongodb+srv://" + mongo_user + ":" + mongo_pw + "@" + mongo_host + "/?retryWrites=true&w=majority&appName=" + mongo_app)


DUDA:

In [ ]:
conn = pymongo.MongoClient("mongodb+srv://" + mongo_user + ":" + mongo_pw + "@" + mongo_host + "/?retryWrites=true&w=majority&appName=" + mongo_app)

Conectando.

In [ ]:
conn.list_database_names()

['sample_mflix', 'admin', 'local']

In [ ]:
print(conn)

MongoClient(host=['ac-vc8fosh-shard-00-02.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-00.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-01.v3hmqat.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterBD2', authsource='admin', replicaset='atlas-m24u8l-shard-0', tls=True)


É necessário salvar a conexão em uma variável. obs.: Quando for testar **NÃO** esquecer de **COLOCAR** o nome do banco.

In [ ]:
bd = conn["e-commerce"]

# ▶ Criando a coleção 'cliente' com validador de esquema.

- Cliente tem sua própria collection pois não seria apropriado o embutir em nenhuma outra,, por conta do contexto.

- Endereço foi embutido em cliente para facilitar buscas como "todos os endereços de cliente X". Como não é esperado que um cliente tenha dúzias de endereços diferentes a restrição de tamanho não se provou relevante.

In [ ]:
bd.create_collection("cliente", validator= {
    "$jsonSchema": {
        "bsonType":"object",
        "title": "Validacao de esquema",
        "required": ["nome", "cpf", "email", "endereco"],
        "properties": {
            "nome": {
                "bsonType": "string",
                "description": "Nome do cliente - Campo obrigatório do tipo string!"
            },
            "cpf": {
                "bsonType": "string",
                "description": "Cpf do cliente - Campo obrigatório do tipo string!"
            },
            "email": {
                "bsonType": "string",
                "description": "Email do cliente - Campo obrigatório do tipo string!"
            },
            "endereco": {
                "bsonType":"array",
                "required": ["_id","cep", "numero"],
                "properties": {
                    "_id":{
                      "bsonType": "int",
                      "description": ""
                    },
                    "cep": {
                        "bsonType":"string",
                        "description": "CEP - Campo obrigatório do tipo string!"
                    },
                    "numero": {
                        "bsonType":"int",
                        "description": "Número do endereço - Campo obrigatório do tipo string!"
                    },
                    "complemento": {
                        "bsonType":"string",
                        "description": "Complemento do endereço - Campo opcional do tipo string!"
                    }
                }
            }
        }
    }
})

Collection(Database(MongoClient(host=['ac-vc8fosh-shard-00-02.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-00.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-01.v3hmqat.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterBD2', authsource='admin', replicaset='atlas-m24u8l-shard-0', tls=True), 'e-commerce'), 'cliente')

# ▶ Criando a coleção 'categoria' com validador de esquema.

In [ ]:
bd.create_collection("categoria", validator= {
    "$jsonSchema": {
        "bsonType":"object",
        "title": "Validacao de esquema",
        "required": ["nome"],
        "properties": {
            "nome": {
                "bsonType": "string",
                "description": "Nome da categoria - Campo obrigatório do tipo string!"
            }
        }
    }
})

Collection(Database(MongoClient(host=['ac-vc8fosh-shard-00-02.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-00.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-01.v3hmqat.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterBD2', authsource='admin', replicaset='atlas-m24u8l-shard-0', tls=True), 'e-commerce'), 'categoria')

# ▶ Criando a coleção 'produto' com validador de esquema.

- A coleção Produto não foi embutida em Categoria pois produtos tendem a ser atulizados frequentemente, o que poderia ser dificultado já que todo o documento da categoria seria modificado. Adicionalmente, como se trata de um E-Commerce, é esperado que cada categoria tenha centanas-milhares de produtos fazendo com que a restrição de espaço se torne um impecilho.
- A coleção Avaliação foi embutida em produto para facilitar buscas como "todas as avaliações do produto X".


In [ ]:
bd.create_collection("produto", validator= {
    "$jsonSchema": {
        "bsonType":"object",
        "title": "Validacao de esquema",
        "required": ["nome", "preco", "id_categoria"],
        "properties": {
            "nome": {
                "bsonType": "string",
                "description": "Nome do produto - Campo obrigatório do tipo string!"
            },
            "preco": {
                "bsonType": "double",
                "description": "Preco do produto - Campo obrigatório do tipo double!"
            },
            "id_categoria": {
                "bsonType": "objectId",
                "description": "Id da categoria do produto - Campo obrigatório do tipo ObjectId!"
            },
            "avaliacao": {
                "bsonType": "array",
                "required": ["id_cliente", "nota"],
                "properties": {
                    "id_cliente": {
                        "bsonType": "objectId",
                        "description": "Id do cliente - Campo obrigatório do tipo objectId!"
                    },
                    "nota": {
                        "bsonType": "int",
                        "description": "Nota da avaliação - Campo obrigatório do tipo int!"
                    },
                    "comentario": {
                        "bsonType": "string",
                        "description": "Comentário da avaliação - Campo opcional do tipo string!"
                    }
                }
            }
        }
    }
})

Collection(Database(MongoClient(host=['ac-vc8fosh-shard-00-02.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-00.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-01.v3hmqat.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterBD2', authsource='admin', replicaset='atlas-m24u8l-shard-0', tls=True), 'e-commerce'), 'produto')

# ▶ Criando a coleção 'transportadora' com validador de esquema.

- Transportadora foi mantida como coleção independente por conta da cardinalidade dela com Item, o que faz impossível embuti-la em Item.

In [ ]:
bd.create_collection("transportadora", validator= {
    "$jsonSchema": {
        "bsonType":"object",
        "title": "Validacao de esquema",
        "required": ["nome", "email"],
        "properties": {
            "nome": {
                "bsonType": "string",
                "description": "Nome da transportadora - Campo obrigatório do tipo string!"
            },
            "email": {
                "bsonType": "string",
                "description": "Email - Campo obrigatório do tipo string!"
            }
        }
    }
})

Collection(Database(MongoClient(host=['ac-vc8fosh-shard-00-02.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-00.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-01.v3hmqat.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterBD2', authsource='admin', replicaset='atlas-m24u8l-shard-0', tls=True), 'e-commerce'), 'transportadora')

# ▶ Criando a coleção 'pedido' com validador de esquema.

- Foi embutido na coleção Pedido a coleção Item. Pelo contexto foi deicidido manter a coleção Pedido independente.

In [ ]:
bd.create_collection("pedido", validator= {
    "$jsonSchema": {
        "bsonType":"object",
        "title": "Validacao de esquema",
        "required": ["data_hora", "forma_pgto", "item", "id_cliente", "id_endereco"],
        "properties": {
            "data_hora": {
                "bsonType": "date",
                "description": "Momento em que foi efetuado o pedido - Campo obrigatório do tipo timestamp!"
            },
            "forma_pgto": {
                "bsonType": "string",
                "description": "Forma de pagamento - Campo obrigatório do tipo string!"
            },
            "item": {
                "bsonType": "array",
                "required": ["id_produto", "quantidade", "data_entrega", "status", "id_transportadora"],
                "properties": {
                    "id_produto": {
                        "bsonType": "objectId",
                        "description": "Id do produto - Campo obrigatório do tipo objectId!"
                    },
                    "quantidade": {
                        "bsonType": "int",
                        "description": "Quantidade - Campo obrigatório do tipo int!"
                    },
                    "data_entrega": {
                        "bsonType": "date",
                        "description": "Data prevista para entrega - Campo obrigatório do tipo date!"
                    },
                    "status": {
                        "bsonType": "string",
                        "description": "Status da compra - Cmapo obrigatório do tipo string!"
                    },
                    "id_transportadora": {
                        "bsonType": "objectId",
                        "description": "Id da transportadora - Campo obrigatório do tipo objectId!"
                    }
                }
            },
            "id_cliente": {
                "bsonType": "objectId",
                "description": "Id do cliente - Campo obrigatório do tipo objetcId!"
            },
            "id_endereco": {
                "bsonType": "int",
                "description": "Id do endereco - Campo obrigatório do tipo int!"
            }
        }
    }
})

Collection(Database(MongoClient(host=['ac-vc8fosh-shard-00-02.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-00.v3hmqat.mongodb.net:27017', 'ac-vc8fosh-shard-00-01.v3hmqat.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterBD2', authsource='admin', replicaset='atlas-m24u8l-shard-0', tls=True), 'e-commerce'), 'pedido')

# ▶ Inserts nas collections

- Insert clientes



In [ ]:
clientes = [
    {
        "nome": "Aurora Celeste Amaral",
        "cpf": "12345678900",
        "email": "aurora.celeste.amaral@ex.com",
        "endereco": [
            {
                "_id": "0",
                "cep": "12345678",
                "numero": 100,
                "complemento": "Apto 101"
            },
            {
                "_id": "1",
                "cep": "67890123",
                "numero": 60,
                "complemento": "Casa 2"
            }
        ]
    },
    {
        "nome": "Dante Ícaro Monteiro",
        "cpf": "23456789011",
        "email": "dante.icaro.monteiro@exe.com",
        "endereco": [
            {
                "_id": "2",
                "cep": "23456789",
                "numero": 236,
                "complemento": None
            },
            {
                "_id": "3",
                "cep": "78901234",
                "numero": 7,
                "complemento": "Ap 404 Bloco 2"
            }
        ]
    },
    {
        "nome": "Isadora Lívia Pereira",
        "cpf": "34567890122",
        "email": "isadora.livia.pereira@ele.com",
        "endereco": [
            {
                "_id": "4",
                "cep": "34567890",
                "numero": 350,
                "complemento": "Apto 202"
            }
        ]
    },
    {
        "nome": "Valentina Sofia Barros",
        "cpf": "45678901233",
        "email": "valentina.sofia.barros@ele.com",
        "endereco": [
            {
              "_id": "5",
              "cep": "45678901",
              "numero": 407,
              "complemento": "Bloco B"
          }
        ]
    },
    {
        "nome": "Leandro Tiago Fernandes",
        "cpf": "56789012344",
        "email": "leandro.tiago.fernandes@emle.com",
        "endereco" : [
            {
                "_id": "6",
                "cep": "56789012",
                "numero": 21,
                "complemento": "Ap 339 Bl D"
            },
            {
                "_id": "7",
                "cep": "89012345",
                "numero": 88,
                "complemento": "Ap 201 Bloco C"
            }
        ]
    }
]

In [ ]:
try:
    idClientes = bd.cliente.insert_many(clientes).inserted_ids
    print(idClientes)
except Exception as e:
    print(f"Erro completo: {e}")
    print(f"Tipo de erro: " + type(e).__name__)
    print(f"Código: {e.details.get('code')}")
    print(f"Mensagem: {e.details.get('errmsg')}")

[ObjectId('666f03e41d6e9b49c4f7debf'), ObjectId('666f03e41d6e9b49c4f7dec0'), ObjectId('666f03e41d6e9b49c4f7dec1'), ObjectId('666f03e41d6e9b49c4f7dec2'), ObjectId('666f03e41d6e9b49c4f7dec3')]



- Insert categorias

In [ ]:
categorias = [
    {
        "nome": "Casa e Cozinha"
    },
    {
        "nome": "Informática"
    },
    {
        "nome": "Papelaria e Escritório"
    }
]

In [ ]:
try:
    idCategorias = bd.categoria.insert_many(categorias).inserted_ids
except Exception as e:
    print(f"Erro completo: {e}")
    print(f"Tipo de erro: " + type(e).__name__)
    print(f"Código: {e.details.get('code')}")
    print(f"Mensagem: {e.details.get('errmsg')}")

- Insert produtos

In [ ]:
produtos = [
    {
        "nome": "Panela de Pressão",
        "preco": 199.9,
        "id_categoria": idCategorias[0],
        "avaliacao": [
            {
                "id_cliente": idClientes[0],
                "nota": 4,
                "comentario": "Amei! Só que chegou atrasado... culpa do correio."
            }
        ]
    },
    {
        "nome": "Jogo de Talheres",
        "preco": 89.99,
        "id_categoria": idCategorias[0],
        "avaliacao": [
            {
                "id_cliente": idClientes[0],
                "nota": 4,
                "comentario": "Mais ou menos. Esperava mais pelo preço."
            }
        ]
    },
    {
        "nome": "Liquidificador",
        "preco": 129.9,
        "id_categoria": idCategorias[0],
        "avaliacao": [
            {
                "id_cliente": idClientes[1],
                "nota": 4,
                "comentario": "Gostei, mas poderia ser mais barato."
            }
        ]
    },
    {
        "nome": "Faqueiro Inox",
        "preco": 249.9,
        "id_categoria": idCategorias[0],
        "avaliacao": [
            {
                "id_cliente": idClientes[1],
                "nota": 4,
                "comentario": "Bom produto, mas a embalagem estava rasgada."
            }
        ]
    },
    {
        "nome": "Notebook",
        "preco": 3599.9,
        "id_categoria": idCategorias[1],
        "avaliacao":[
            {
                "id_cliente": idClientes[2],
                "nota": 5,
                "comentario": "Perfeito! Já indiquei para os amigos."
            },
            {
                "id_cliente": idClientes[3],
                "nota": 5,
                "comentario": "OTMIO CONSIGO JOGAR GENSHIN O DIA TOODO:)"
            }
        ]
    },
    {
        "nome": "Mouse Gamer",
        "preco": 149.9,
        "id_categoria": idCategorias[1],
        "avaliacao": [
            {
                "id_cliente": idClientes[0],
                "nota": 5,
                "comentario": "Fantástico! Minha avó adorou."
            },
            {
                "id_cliente": idClientes[4],
                "nota": 2,
                "comentario": "Não gostei, mas o meu gato adorou..."
           }
        ]
    },
    {
        "nome": "Teclado Mecânico",
        "preco": 299.9,
        "id_categoria": idCategorias[1],
        "avaliacao": [
            {
                "id_cliente": idClientes[1],
                "nota": 3,
                "comentario": "Chegou rápido, lindo, siplesmente INCRÍVEL"
            },
            {
                "id_cliente": idClientes[4],
                "nota": 5,
                "comentario": "Muito bonito, funciona perfeitamente"
            }
        ]
    },
    {
        "nome": "Monitor 24\"",
        "preco": 899.9,
        "id_categoria": idCategorias[1],
        "avaliacao": [
            {
                "id_cliente": idClientes[1],
                "nota": 5,
                "comentario": "Simplesmente incrível! Até meu vizinho quer um agora."
            },
            {
                "id_cliente": idClientes[2],
                "nota": 3,
                "comentario": "É okay, nada de espetacular."
            }
        ]
    },
    {
        "nome": "Caderno Universitário",
        "preco": 19.9,
        "id_categoria": idCategorias[2],
        "avaliacao": [
            {
                "id_cliente": idClientes[2],
                "nota": 1,
                "comentario": "Veio todo amassado... Decepção total."
           },
           {
               "id_cliente": idClientes[3],
               "nota": 5,
               "comentario": "Produto maravilhiso! Recomendo pra todo mundo."
          }
        ]
    },
    {
        "nome": "Caneta Esferográfica",
        "preco": 2.99,
        "id_categoria": idCategorias[2],
        "avaliacao": [
            {
                "id_cliente": idClientes[2],
                "nota": 4,
                "comentario": "Bom, mas o manual veio em chinês."
            },
            {
                "id_cliente": idClientes[4],
                "nota": 1,
                "comentario": "Detestei! Veio completamente diferente do anunciado."
            },
            {
                "id_cliente": idClientes[3],
                "nota": 5,
                "comentario": "Escreve"
            }
        ]
    },
    {
        "nome": "Agenda 2024",
        "preco": 39.9,
        "id_categoria": idCategorias[2],
        "avaliacao": [
            {
                "id_cliente": idClientes[3],
                "nota": 3,
                "comentario": "Horrivel! Não funcona de jeito nenhum."
            },
            {
                "id_cliente": idClientes[4],
                "nota": 5,
                "comentario": "Muito bonito e organizado AMEEI"
            }
        ]
    },
    {
        "nome": "Grampeador",
        "preco": 25.9,
        "id_categoria": idCategorias[2],
        "avaliacao": [
            {
                "id_cliente": idClientes[0],
                "nota": 5,
                "comentario": "Superou minhas expectativas! Vou comprar mais."
            }
        ]
    }
]

In [ ]:
try:
    idProdutos = bd.produto.insert_many(produtos).inserted_ids
except Exception as e:
    print(f"Erro completo: {e}")
    print(f"Tipo de erro: " + type(e).__name__)
    print(f"Código: {e.details.get('code')}")
    print(f"Mensagem: {e.details.get('errmsg')}")

- Insert transportadoras

In [ ]:
transportadoras = [
    {
        "nome": "Transporte Águia Dourada",
        "email": "info@aguiadourada.com"
    },
    {
        "nome": "Entrega Expressa",
        "email": "suporte@entregaexpressa.com"
    }
]

In [ ]:
try:
    idTrans = bd.transportadora.insert_many(transportadoras).inserted_ids
except Exception as e:
    print(f"Erro completo: {e}")
    print(f"Tipo de erro: " + type(e).__name__)
    print(f"Código: {e.details.get('code')}")
    print(f"Mensagem: {e.details.get('errmsg')}")

- Insert pedidos

In [ ]:
pedidos = [
    {
        "data_hora": datetime(2024,1,1,10,15,30),
        "forma_pgto": "CC",
        "id_cliente": idClientes[0],
        "id_endereco": 0,
        "item": [
            {
                "id_produto": idProdutos[0],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-01-09', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            },
            {
                "id_produto": idProdutos[1],
                "quantidade": 2,
                "data_entrega": datetime.strptime('2024-01-06', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            }

        ]
    },
    {
        "data_hora": datetime(2024,1,15,14,20,45),
        "forma_pgto": "BO",
        "id_cliente": idClientes[1],
        "id_endereco": 2,
        "item": [
             {
                 "id_produto": idProdutos[2],
                 "quantidade": 1,
                 "data_entrega": datetime.strptime('2024-01-20', '%Y-%m-%d'),
                 "status": "ENTREGUE",
                 "id_transportadora": idTrans[0]
             },
             {
                 "id_produto": idProdutos[3],
                 "quantidade": 1,
                 "data_entrega": datetime.strptime('2024-01-20', '%Y-%m-%d'),
                 "status": "ENTREGUE",
                 "id_transportadora": idTrans[0]
             }

        ]
    },
    {
        "data_hora": datetime(2024,2,17,16,35,50),
        "forma_pgto": "PX",
        "id_cliente": idClientes[2],
        "id_endereco": 4,
        "item": [
            {
                "id_produto": idProdutos[8],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-02-26', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            },
            {
                "id_produto": idProdutos[9],
                "quantidade": 2,
                "data_entrega": datetime.strptime('2024-02-26', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            }
        ]
    },
    {
        "data_hora": datetime(2024,3,4,11,45,30),
        "forma_pgto": "CD",
        "id_cliente": idClientes[3],
        "id_endereco": 5,
        "item": [
            {
                "id_produto": idProdutos[10],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-03-12', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            },
            {
                "id_produto": idProdutos[8],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-03-10', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            }
        ]
    },
    {
        "data_hora": datetime(2024,3,5,9,10,25),
        "forma_pgto": "PX",
        "id_cliente": idClientes[4],
        "id_endereco": 6,
        "item": [
            {
                "id_produto": idProdutos[5],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-03-11', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            },
            {
                "id_produto": idProdutos[9],
                "quantidade": 5,
                "data_entrega": datetime.strptime('2024-03-14', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            }
        ]
    },
    {
        "data_hora": datetime(2024,3,6,17,25,30),
        "forma_pgto": "CC",
        "id_cliente": idClientes[0],
        "id_endereco": 1,
        "item": [
            {
                "id_produto": idProdutos[5],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-03-17', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            },
            { "id_produto": idProdutos[11],
              "quantidade": 1,
              "data_entrega": datetime.strptime('2024-03-17', '%Y-%m-%d'),
              "status": "ENTREGUE",
              "id_transportadora": idTrans[1]
            }
        ]
    },
    {
        "data_hora": datetime(2024,4,20,12,50,40),
        "forma_pgto": "BO",
        "id_cliente": idClientes[1],
        "id_endereco": 3,
        "item": [
            {
                "id_produto": idProdutos[6],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-05-01', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            },
            {
                "id_produto": idProdutos[7],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-05-01', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            }
        ]
    },
    {
        "data_hora": datetime(2024,4,26,18,15,20),
        "forma_pgto": "PX",
        "id_cliente": idClientes[2],
        "id_endereco": 4,
        "item": [
            {
                "id_produto": idProdutos[4],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-05-10', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            },
            {
                "id_produto": idProdutos[7],
                "quantidade": 2,
                "data_entrega": datetime.strptime('2024-05-10', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[0]
            }
        ]
    },
    {
        "data_hora": datetime(2024,5,1,15,30,55),
        "forma_pgto": "CD",
        "id_cliente": idClientes[3],
        "id_endereco": 5,
        "item": [
            {
                "id_produto": idProdutos[4],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-05-15', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            },
            {
                "id_produto": idProdutos[9],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-05-15', '%Y-%m-%d'),
                "status": "ENTREGUE",
                "id_transportadora": idTrans[1]
            }
        ]
    },
    {
        "data_hora": datetime(2024,5,19,20,45,35),
        "forma_pgto": "CD",
        "id_cliente": idClientes[4],
        "id_endereco": 7,
        "item": [
            {
                "id_produto": idProdutos[10],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-06-20', '%Y-%m-%d'),
                "status": "TRANSITO",
                "id_transportadora": idTrans[0]
            },
            {
                "id_produto": idProdutos[6],
                "quantidade": 1,
                "data_entrega": datetime.strptime('2024-06-20', '%Y-%m-%d'),
                "status": "TRANSITO",
                "id_transportadora": idTrans[1]
            }
        ]
    }
]

In [ ]:
try:
    bd.pedido.insert_many(pedidos)
except Exception as e:
    print(f"Erro completo: {e}")
    print(f"Tipo de erro: " + type(e).__name__)
    print(f"Código: {e.details.get('code')}")
    print(f"Mensagem: {e.details.get('errmsg')}")

# ▶ Selects


- Média de avaliações por categoria

In [ ]:
result = bd.produto.aggregate([
    {"$unwind": "$avaliacao"},
    {"$lookup": {
        "from": "categoria",
        "localField": "id_categoria",
        "foreignField": "_id",
        "as": "categoria"
    }},
    {"$unwind": "$categoria"},
    {"$group": {
        "_id": "$categoria.nome",
        "Media_Nota": {"$avg": "$avaliacao.nota"},
        "Media_Precos": {"$avg": "$preco"}
    }},
    {"$project": {
        "_id": 0,
        "nome": "$_id",
        "Media_Nota": 1,
        "Media_Precos": 1
    }}
])
for r in result:
  print(f"Categoria: {r['nome']} | Média avaliações: {round(r['Media_Nota'], 1)} | Média preços: {round(r['Media_Precos'], 2)}")

Categoria: Casa e Cozinha | Média avaliações: 4.0 | Média preços: 167.42
Categoria: Papelaria e Escritório | Média avaliações: 3.6 | Média preços: 19.31
Categoria: Informática | Média avaliações: 4.1 | Média preços: 1237.4


- Média de avaliações por produto

In [ ]:
result = bd.produto.aggregate([
    {"$unwind": "$avaliacao"},
    {"$group": {
        "_id": {"nome": "$nome", "preco": "$preco"},
        "Media_Nota": {"$avg": "$avaliacao.nota"}
    }},
    {"$project": {
        "nome": "$_id.nome",
        "preco": "$_id.preco",
        "Media_Nota": 1
    }}
])
for r in result:
    print(f"Produto: {r['nome']} | Nota: {round(r['Media_Nota'], 1)} | Preço: {round(r['preco'], 2)}")

Produto: Faqueiro Inox | Nota: 4.0 | Preço: 249.9
Produto: Monitor 24" | Nota: 4.0 | Preço: 899.9
Produto: Agenda 2024 | Nota: 4.0 | Preço: 39.9
Produto: Liquidificador | Nota: 4.0 | Preço: 129.9
Produto: Jogo de Talheres | Nota: 4.0 | Preço: 89.99
Produto: Caderno Universitário | Nota: 3.0 | Preço: 19.9
Produto: Caneta Esferográfica | Nota: 3.3 | Preço: 2.99
Produto: Panela de Pressão | Nota: 4.0 | Preço: 199.9
Produto: Notebook | Nota: 5.0 | Preço: 3599.9
Produto: Mouse Gamer | Nota: 3.5 | Preço: 149.9
Produto: Teclado Mecânico | Nota: 4.0 | Preço: 299.9
Produto: Grampeador | Nota: 5.0 | Preço: 25.9


- Avaliações de todos os produtos

In [ ]:
result = bd.produto.aggregate([{"$unwind": "$avaliacao"},
    {"$lookup": {
        "from": "categoria",
        "localField": "id_categoria",
        "foreignField": "_id",
        "as": "categoria"
    }},
    {"$unwind": "$categoria"},
    {"$project": {
        "_id": 0,
        "nota": "$avaliacao.nota",
        "comentario": "$avaliacao.comentario",
        "produto": "$nome",
        "preco": "$preco",
        "categoria": "$categoria.nome"
    }}])
for r in result:
    print(f"Produto: {r['produto']} | Nota: {round(r['nota'], 1)} | Preço: {round(r['preco'], 2)} | Comentário: {r['comentario']} | Categoria: {r['categoria']}")

Produto: Panela de Pressão | Nota: 4 | Preço: 199.9 | Comentário: Amei! Só que chegou atrasado... culpa do correio. | Categoria: Casa e Cozinha
Produto: Jogo de Talheres | Nota: 4 | Preço: 89.99 | Comentário: Mais ou menos. Esperava mais pelo preço. | Categoria: Casa e Cozinha
Produto: Liquidificador | Nota: 4 | Preço: 129.9 | Comentário: Gostei, mas poderia ser mais barato. | Categoria: Casa e Cozinha
Produto: Faqueiro Inox | Nota: 4 | Preço: 249.9 | Comentário: Bom produto, mas a embalagem estava rasgada. | Categoria: Casa e Cozinha
Produto: Notebook | Nota: 5 | Preço: 3599.9 | Comentário: Perfeito! Já indiquei para os amigos. | Categoria: Informática
Produto: Notebook | Nota: 5 | Preço: 3599.9 | Comentário: OTMIO CONSIGO JOGAR GENSHIN O DIA TOODO:) | Categoria: Informática
Produto: Mouse Gamer | Nota: 5 | Preço: 149.9 | Comentário: Fantástico! Minha avó adorou. | Categoria: Informática
Produto: Mouse Gamer | Nota: 2 | Preço: 149.9 | Comentário: Não gostei, mas o meu gato adorou... |

- 5 produtos mais vendidos

In [ ]:
top_vendidos = bd.pedido.aggregate([
    {"$unwind": "$item"},
    {"$group": {
        "_id": "$item.id_produto",
        "qtd_vendidos": {"$sum": "$item.quantidade"}
      }
    },
    {"$sort": {"qtd_vendidos": -1}},
    {"$limit": 5},
    {"$lookup": {
        "from": "produto",
        "localField": "_id",
        "foreignField": "_id",
        "as": "prod"
      }
    },
    {"$unwind": "$prod"},
    {"$project": {
        "_id": 1,
        "qtd_vendidos": 1,
        "prod.nome": 1
    }}
])

for tp in top_vendidos:
    print(f"Produto: {tp['prod']['nome']} | QTD_Vendidos: {tp['qtd_vendidos']}")

Produto: Caneta Esferográfica | QTD_Vendidos: 8
Produto: Monitor 24" | QTD_Vendidos: 3
Produto: Notebook | QTD_Vendidos: 2
Produto: Teclado Mecânico | QTD_Vendidos: 2
Produto: Caderno Universitário | QTD_Vendidos: 2
